# Libraries 

In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd

import numpy as np

import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns 

sns.set(color_codes = True)

%matplotlib inline

import plotly.graph_objects as go

from plotly.subplots import make_subplots

from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter

from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

from textblob import Word

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score, KFold

from textblob import TextBlob

import gender_guesser.detector as gender

import nltk

from nltk.corpus import stopwords

import re

import emoji

from scipy.stats import chi2_contingency

from matplotlib.patches import ConnectionPatch

from langdetect import detect

from nltk.probability import *

from nltk.tokenize import word_tokenize

from nltk import pos_tag

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer

import plotly.graph_objects as go

from plotly.subplots import make_subplots

from afinn import Afinn

from sklearn.model_selection import cross_val_score, KFold

from sklearn.ensemble import RandomForestClassifier

from nltk.text import Text


import string

from collections import Counter

from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC

from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

In [3]:
#pip install emoji

In [4]:
#pip install genderize

In [5]:
#!pip install gender-guesser

In [6]:
#pip install --upgrade seaborn pandas

In [7]:
#pip install langdetect

In [8]:
#pip install afinn

In [9]:
#pip install wordcloud

### Color Pallete Brand Kit

"#ff914d"  -- Orange
"#1a562a"  -- Green

# Raw Data

In [10]:
ml = pd.read_csv("ml.csv")

In [11]:
ml.head()

,starRating,Gender_Numeric,Year,cleaned_text,sentiment_score,word_count,avg_word,sentiment_label,sentiment_ml,sentiment
0,5,1,2019,im every beginning year highly recommend chale...,0.030000,11,6.272727,Neutral,1,0
1,5,2,2019,beautiful pleasant place breakfast lunch lot v...,0.861111,8,6.875000,Positive,2,0
2,1,2,2019,thought horrible location much many demand own...,-0.050000,15,5.466667,Negative,0,0
3,5,1,2019,one best canpings region,1.000000,4,5.250000,Positive,2,0
4,5,1,2019,family place found thing site expensive like l...,-0.500000,12,5.416667,Negative,0,0


In [12]:
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648 entries, 0 to 1647
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   starRating       1648 non-null   int64  
 1   Gender_Numeric   1648 non-null   int64  
 2   Year             1648 non-null   int64  
 3   cleaned_text     1645 non-null   object 
 4   sentiment_score  1648 non-null   float64
 5   word_count       1648 non-null   int64  
 6   avg_word         1648 non-null   float64
 7   sentiment_label  1648 non-null   object 
 8   sentiment_ml     1648 non-null   int64  
 9   sentiment        1648 non-null   int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 128.9+ KB


In [13]:
ml["cleaned_text"].isnull().sum()

3

In [14]:
ml["cleaned_text"].isnull().sum()

3

In [15]:
ml.dropna(subset = ["cleaned_text"], inplace = True)

In [16]:
ml["cleaned_text"].isnull().sum()

0

# Machine Learning 

Stratified Splitting:

The stratify=y argument is crucial for classification problems where you have multiple categories in your target variable y. It ensures that the proportion of classes in the training set is similar to the proportion in the entire dataset. This is important to avoid a situation where the training set might be biased towards certain classes, leading to poor model performance on the unseen test data.

https://scikit-learn.org/stable/modules/cross_validation.html#stratification

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)  # Keep all features
X = tfidf.fit_transform(ml["cleaned_text"])


In [17]:
tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X = tfidf.fit_transform(ml["cleaned_text"])
y = ml["sentiment_ml"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, 
                                                    random_state = 12, stratify = y)


In [18]:
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('MNB', MultinomialNB()))
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(max_iter = 1000)))
models.append(('LSVC', LinearSVC(random_state = 12)))

In [19]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state = 12, shuffle = True)
	cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

RF: 0.832813 (0.020497)
MNB: 0.794795 (0.035304)
XGB: 0.854869 (0.017760)
LR: 0.848791 (0.024575)
LSVC: 0.856425 (0.021318)


In [20]:
results = []
names = []
class_reports = []

for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)  
    cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    class_report_fold = classification_report(y_test, y_pred)
    class_reports.append(class_report_fold)
    results.append(cv_results)
    names.append(name)

    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    print(f"\nClassification Report for {name}:\n{class_report_fold}\n")

RF: Mean Accuracy - 0.8237, Std. Deviation - 0.0240

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.14      0.25        14
           1       0.77      0.90      0.83       140
           2       0.91      0.86      0.88       175

    accuracy                           0.84       329
   macro avg       0.90      0.63      0.66       329
weighted avg       0.86      0.84      0.84       329


MNB: Mean Accuracy - 0.7903, Std. Deviation - 0.0316

Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.80      0.70      0.75       140
           2       0.79      0.94      0.86       175

    accuracy                           0.80       329
   macro avg       0.53      0.55      0.54       329
weighted avg       0.76      0.80      0.78       329


XGB: Mean Accuracy - 0.8511, Std. Deviation - 0.0293

Classification

# Under Sampling

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X = tfidf.fit_transform(ml["cleaned_text"])
y = ml["sentiment_ml"]

from imblearn.under_sampling import NearMiss

nr = NearMiss()

X_train, y_train = nr.fit_resample(X_train, y_train)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, 
                                                    random_state = 12, stratify = y)


In [105]:
from imblearn.under_sampling import NearMiss

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12, stratify=y)

tfidf = TfidfVectorizer(ngram_range=(1, 1), max_features=500, tokenizer=word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nr = NearMiss()
X_train_nm, y_train_nm = nr.fit_resample(X_train_tfidf, y_train)

In [106]:
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('MNB', MultinomialNB()))
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('LSVC', LinearSVC(random_state=12)))

In [107]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
	cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = 'accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

RF: 0.650000 (0.089646)
MNB: 0.573856 (0.089940)
XGB: 0.737255 (0.138704)
LR: 0.585621 (0.120489)
LSVC: 0.591503 (0.124706)


In [108]:
results = []
names = []
class_reports = []

for name, model in models:
    kfold = StratifiedKFold(n_splits = 5, random_state = 1, shuffle = True)
    model.fit(X_train_nm, y_train_nm)  
    y_pred = model.predict(X_test_tfidf)  
    cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = "accuracy")
    class_report_fold = classification_report(y_test, y_pred)
    class_reports.append(class_report_fold)
    results.append(cv_results)
    names.append(name)

    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    print(f"\nClassification Report for {name}:\n{class_report_fold}\n")

RF: Mean Accuracy - 0.6313, Std. Deviation - 0.0497

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.21      0.50      0.30        14
           1       0.59      0.65      0.62       140
           2       0.81      0.65      0.72       175

    accuracy                           0.64       329
   macro avg       0.54      0.60      0.55       329
weighted avg       0.69      0.64      0.66       329


MNB: Mean Accuracy - 0.5845, Std. Deviation - 0.0367

Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.21      0.79      0.33        14
           1       0.58      0.30      0.40       140
           2       0.71      0.83      0.76       175

    accuracy                           0.60       329
   macro avg       0.50      0.64      0.50       329
weighted avg       0.63      0.60      0.59       329


XGB: Mean Accuracy - 0.7134, Std. Deviation - 0.0287

Classification

# Over Sampling

In [47]:
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y, test_size = 0.30)

smt = SMOTE()

X_train, y_train = smt.fit_resample(X_train, y_train)

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X = tfidf.fit_transform(ml["cleaned_text"])
y = ml["sentiment_ml"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, 
                                                    random_state = 12, stratify = y)


In [48]:
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('MNB', MultinomialNB()))
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('LSVC', LinearSVC(random_state=12)))

In [49]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=12, shuffle=True)
	cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

RF: 0.893273 (0.022617)
MNB: 0.850232 (0.025954)
XGB: 0.905236 (0.026296)
LR: 0.895994 (0.018676)
LSVC: 0.897081 (0.022971)


In [50]:
results = []
names = []
class_reports = []

for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)  
    cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    class_report_fold = classification_report(y_test, y_pred)
    class_reports.append(class_report_fold)
    results.append(cv_results)
    names.append(name)

    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    print(f"\nClassification Report for {name}:\n{class_report_fold}\n")

RF: Mean Accuracy - 0.8954, Std. Deviation - 0.0257

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        21
           1       0.80      0.83      0.81       210
           2       0.90      0.86      0.88       263

    accuracy                           0.84       494
   macro avg       0.75      0.76      0.76       494
weighted avg       0.84      0.84      0.84       494


MNB: Mean Accuracy - 0.8540, Std. Deviation - 0.0241

Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.26      0.67      0.37        21
           1       0.76      0.64      0.70       210
           2       0.84      0.84      0.84       263

    accuracy                           0.75       494
   macro avg       0.62      0.72      0.64       494
weighted avg       0.78      0.75      0.76       494


XGB: Mean Accuracy - 0.8998, Std. Deviation - 0.0297

Classification

# n gram 2

In [56]:
tfidf = TfidfVectorizer(ngram_range = (1, 2), max_features = 500, tokenizer = word_tokenize)
X = tfidf.fit_transform(ml["cleaned_text"])
y = ml["sentiment_ml"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, 
                                                    random_state = 12, stratify = y)


In [57]:
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('MNB', MultinomialNB()))
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(max_iter = 1000)))
models.append(('LSVC', LinearSVC(random_state = 12)))

In [58]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state = 12, shuffle = True)
	cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

RF: 0.831344 (0.024009)
MNB: 0.786462 (0.033209)
XGB: 0.841204 (0.027273)
LR: 0.840458 (0.019558)
LSVC: 0.853372 (0.018728)


In [59]:
results = []
names = []
class_reports = []

for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)  
    cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    class_report_fold = classification_report(y_test, y_pred)
    class_reports.append(class_report_fold)
    results.append(cv_results)
    names.append(name)

    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    print(f"\nClassification Report for {name}:\n{class_report_fold}\n")

RF: Mean Accuracy - 0.8268, Std. Deviation - 0.0239

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.21      0.35        14
           1       0.79      0.91      0.85       140
           2       0.93      0.87      0.90       175

    accuracy                           0.86       329
   macro avg       0.91      0.67      0.70       329
weighted avg       0.87      0.86      0.85       329


MNB: Mean Accuracy - 0.7895, Std. Deviation - 0.0349

Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.80      0.73      0.76       140
           2       0.81      0.93      0.86       175

    accuracy                           0.80       329
   macro avg       0.53      0.55      0.54       329
weighted avg       0.77      0.80      0.78       329


XGB: Mean Accuracy - 0.8404, Std. Deviation - 0.0324

Classification

# n gram 3

In [76]:
tfidf = TfidfVectorizer(ngram_range = (1, 3), max_features = 500, tokenizer = word_tokenize)
X = tfidf.fit_transform(ml["cleaned_text"])
y = ml["sentiment_ml"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, 
                                                    random_state = 12, stratify = y)


In [77]:
models = []
models.append(('RF', RandomForestClassifier()))
models.append(('MNB', MultinomialNB()))
models.append(('XGB', XGBClassifier()))
models.append(('LR', LogisticRegression(max_iter = 1000)))
models.append(('LSVC', LinearSVC(random_state = 12)))

In [78]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state = 12, shuffle = True)
	cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = 'accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

RF: 0.828302 (0.034108)
MNB: 0.781905 (0.033114)
XGB: 0.843436 (0.025020)
LR: 0.839700 (0.020265)
LSVC: 0.850324 (0.016501)


In [79]:
results = []
names = []
class_reports = []

for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)  
    cv_results = cross_val_score(model, X_train, y_train, cv = kfold, scoring = "accuracy")
    class_report_fold = classification_report(y_test, y_pred)
    class_reports.append(class_report_fold)
    results.append(cv_results)
    names.append(name)

    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    print(f"\nClassification Report for {name}:\n{class_report_fold}\n")

RF: Mean Accuracy - 0.8366, Std. Deviation - 0.0218

Classification Report for RF:
              precision    recall  f1-score   support

           0       1.00      0.14      0.25        14
           1       0.78      0.89      0.83       140
           2       0.91      0.86      0.89       175

    accuracy                           0.84       329
   macro avg       0.90      0.63      0.66       329
weighted avg       0.86      0.84      0.84       329


MNB: Mean Accuracy - 0.7895, Std. Deviation - 0.0367

Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.79      0.71      0.75       140
           2       0.80      0.92      0.85       175

    accuracy                           0.79       329
   macro avg       0.53      0.54      0.53       329
weighted avg       0.76      0.79      0.77       329


XGB: Mean Accuracy - 0.8381, Std. Deviation - 0.0316

Classification

## Random Forest

In [ ]:
# Build the model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

predicted_rf = rf.predict(X_test)

# Calculate accuracy and print classification report
accuracy_rf = accuracy_score(y_test, predicted_rf)
print('Accuracy:', accuracy_rf)
print('Classification Report:')
print(classification_report(y_test, predicted_rf))

y_train_pred = rf.predict(X_train)

y_test_pred = rf.predict(X_test)

accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Accuracy on Training Set:", accuracy_train)
print("Accuracy on Test Set:", accuracy_test)

# Build confusion matrix
cm_rf = confusion_matrix(y_test, predicted_rf)

# Create heatmap
labels = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm_rf, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted sentiment')
plt.ylabel('True sentiment')
plt.show()

## Naive Bayes Multinominal

In [ ]:
# Build the model
nb = MultinomialNB()
nb.fit(X_train, y_train)

predicted_nb = nb.predict(X_test)

# Calculate accuracy and print classification report 
accuracy_nb = accuracy_score(y_test, predicted_nb)
print('Accuracy:', accuracy_nb)
print('Classification Report:')
print(classification_report(y_test, predicted_nb))


cm_nb = confusion_matrix(y_test, predicted_nb)

# Create heatmap
labels = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm_nb, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted sentiment')
plt.ylabel('True sentiment')
plt.show()

## XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

predicted_xgb = xgb.predict(X_test)

# Calculate accuracy and print classification report 
accuracy_xgb = accuracy_score(y_test, predicted_xgb)
print('Accuracy:', accuracy_xgb)
print('Classification Report:')
print(classification_report(y_test, predicted_xgb))

cm_xgb = confusion_matrix(y_test, predicted_xgb)

# Create heatmap
labels = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm_xgb, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted sentiment')
plt.ylabel('True sentiment')
plt.show()

## Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train)

predicted_lr = lr.predict(X_test)

# Calculate accuracy and print classification report
accuracy_lr = accuracy_score(y_test, predicted_lr)
print('Accuracy:', accuracy_lr)
print('Classification Report:')
print(classification_report(y_test, predicted_lr))

cm_lr = confusion_matrix(y_test, predicted_lr)

# Create heatmap
labels = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm_lr, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted sentiment')
plt.ylabel('True sentiment')
plt.show()

## Linear Support Vector Classification

In [ ]:
svc = LinearSVC(random_state=12)
svc.fit(X_train, y_train)

predicted_svc = svc.predict(X_test)

# Calculate accuracy and print classification report
accuracy_svc = accuracy_score(y_test, predicted_svc)
print('Accuracy:', accuracy_svc)
print('Classification Report:')
print(classification_report(y_test, predicted_svc))


cm_svc = confusion_matrix(y_test, predicted_svc)

# Create heatmap
labels = ['Negative', 'Neutral', 'Positive']
sns.heatmap(cm_svc, annot=True, cmap='Blues', fmt='g', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted sentiment')
plt.ylabel('True sentiment')
plt.show()

In [ ]:
Models = ['Random Forest', 'Naive Bayes Multinominal', 'XGBoost', 'Logistic Regression', 'SVC']
Scores = [accuracy_rf, accuracy_nb, accuracy_xgb, accuracy_lr, accuracy_svc]
performance = pd.DataFrame(list(zip(Models, Scores)), 
                          columns = ['Models', 'Accuracy_score'])\
                            .sort_values('Accuracy_score', ascending=False)
performance

In [ ]:
ml.tail(1)

In [112]:
from imblearn.under_sampling import NearMiss

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 12, stratify = y)

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nr = NearMiss()
X_train_nm, y_train_nm = nr.fit_resample(X_train_tfidf, y_train)

models = [
    ('RF', RandomForestClassifier()),
    ('MNB', MultinomialNB()),
    ('XGB', XGBClassifier()),
    ('LR', LogisticRegression(max_iter = 1000)),
    ('LSVC', LinearSVC())
]


class_reports = {}
for name, model in models:
    kfold = StratifiedKFold(n_splits = 5, random_state = 12, shuffle = True)
    cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = 'accuracy')
    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    
    model.fit(X_train_nm, y_train_nm)
    y_pred = model.predict(X_test_tfidf)
    class_reports[name] = classification_report(y_test, y_pred)

for name, report in class_reports.items():
    print(f"\nClassification Report for {name}:\n{report}\n")

RF: Mean Accuracy - 0.6018, Std. Deviation - 0.1138
MNB: Mean Accuracy - 0.6022, Std. Deviation - 0.0641
XGB: Mean Accuracy - 0.5847, Std. Deviation - 0.0137
LR: Mean Accuracy - 0.6368, Std. Deviation - 0.0975
LSVC: Mean Accuracy - 0.6252, Std. Deviation - 0.0837

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.23      0.79      0.36        14
           1       0.71      0.54      0.62       140
           2       0.81      0.81      0.81       175

    accuracy                           0.70       329
   macro avg       0.59      0.71      0.60       329
weighted avg       0.74      0.70      0.71       329



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.23      0.79      0.36        14
           1       0.77      0.36      0.49       140
           2       0.73      0.90      0.81       175

    accuracy                           0.67       329
   macro avg       

In [113]:
from imblearn.under_sampling import NearMiss

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 12, stratify = y)

tfidf = TfidfVectorizer(ngram_range = (1, 1), max_features = 500, tokenizer = word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nr = NearMiss()
X_train_nm, y_train_nm = nr.fit_resample(X_train_tfidf, y_train)

models = [
    ('RF', RandomForestClassifier()),
    ('MNB', MultinomialNB()),
    ('XGB', XGBClassifier()),
    ('LR', LogisticRegression(max_iter = 1000)),
    ('LSVC', LinearSVC())
]


class_reports = {}
for name, model in models:
    kfold = StratifiedKFold(n_splits = 5, random_state = 12, shuffle = True)
    cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = 'accuracy')
    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    
    model.fit(X_train_nm, y_train_nm)
    y_pred = model.predict(X_test_tfidf)
    class_reports[name] = classification_report(y_test, y_pred)

for name, report in class_reports.items():
    print(f"\nClassification Report for {name}:\n{report}\n")

RF: Mean Accuracy - 0.6923, Std. Deviation - 0.0503
MNB: Mean Accuracy - 0.6049, Std. Deviation - 0.0873
XGB: Mean Accuracy - 0.7031, Std. Deviation - 0.0780
LR: Mean Accuracy - 0.6715, Std. Deviation - 0.0499
LSVC: Mean Accuracy - 0.6350, Std. Deviation - 0.0616

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.45      0.71      0.56         7
           1       0.66      0.77      0.71        70
           2       0.85      0.69      0.76        88

    accuracy                           0.73       165
   macro avg       0.65      0.73      0.68       165
weighted avg       0.75      0.73      0.73       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.36      0.71      0.48         7
           1       0.82      0.46      0.59        70
           2       0.72      0.92      0.81        88

    accuracy                           0.72       165
   macro avg       

In [116]:
from imblearn.under_sampling import NearMiss

X = ml["cleaned_text"]
y = ml["sentiment_ml"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 12, stratify = y)

tfidf = TfidfVectorizer(ngram_range = (1, 3), max_features = 500, tokenizer = word_tokenize)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

nr = NearMiss()
X_train_nm, y_train_nm = nr.fit_resample(X_train_tfidf, y_train)

models = [
    ('RF', RandomForestClassifier()),
    ('MNB', MultinomialNB()),
    ('XGB', XGBClassifier()),
    ('LR', LogisticRegression(max_iter = 500)),
    ('LSVC', LinearSVC())
]


class_reports = {}
for name, model in models:
    kfold = StratifiedKFold(n_splits = 10, random_state = 12, shuffle = True)
    cv_results = cross_val_score(model, X_train_nm, y_train_nm, cv = kfold, scoring = 'accuracy')
    print(f"{name}: Mean Accuracy - {cv_results.mean():.4f}, Std. Deviation - {cv_results.std():.4f}")
    
    model.fit(X_train_nm, y_train_nm)
    y_pred = model.predict(X_test_tfidf)
    class_reports[name] = classification_report(y_test, y_pred)

for name, report in class_reports.items():
    print(f"\nClassification Report for {name}:\n{report}\n")

RF: Mean Accuracy - 0.6397, Std. Deviation - 0.1260
MNB: Mean Accuracy - 0.6153, Std. Deviation - 0.1360
XGB: Mean Accuracy - 0.6676, Std. Deviation - 0.1513
LR: Mean Accuracy - 0.6250, Std. Deviation - 0.1165
LSVC: Mean Accuracy - 0.6047, Std. Deviation - 0.1250

Classification Report for RF:
              precision    recall  f1-score   support

           0       0.29      0.57      0.38         7
           1       0.67      0.61      0.64        70
           2       0.78      0.77      0.78        88

    accuracy                           0.70       165
   macro avg       0.58      0.65      0.60       165
weighted avg       0.71      0.70      0.70       165



Classification Report for MNB:
              precision    recall  f1-score   support

           0       0.22      0.86      0.35         7
           1       0.94      0.43      0.59        70
           2       0.77      0.93      0.85        88

    accuracy                           0.72       165
   macro avg       